In [1]:
!pip install tqdm
!pip install yfinance

In [2]:
import pandas as pd
import numpy as np
import yfinance as yf
from tqdm import tqdm

In [3]:
def get_financial_data(company: list, period: str, interval: str):
    data = yf.download(tickers="SPY AAPL MSFT AMZN",
                       period="1d",
                       interval="1m",
                       group_by='columns',
                       auto_adjust=True,
                       prepost=True,
                       threads=True,
                       proxy=None)
    data = data.loc[:,data.columns.get_level_values(1).isin({"Close"})].droplevel(1, axis=1)
    return data


In [4]:
def sharpe_ratio_portfolio(data: pd.DataFrame, weight: np.array, risk_free_rate: float):
        log_return = np.log(1 + data.pct_change())
        expected_return = np.sum(log_return.mean() * weight * 252)
        expected_volatility = np.sqrt(
                np.dot(
                       weight.T,
                       np.dot(
                               log_return.cov() * 252,
                               weight
                       )
                )
        )
        sharpe = (expected_return - risk_free_rate) / expected_volatility
        return expected_return, expected_volatility, sharpe

In [5]:
def MC_weight_sim(data: pd.DataFrame, risk_free_rate: float, N_sim: int, as_df: bool):
        size_of_portfolio = data.shape[1]
        sim_weights = np.zeros((N_sim, size_of_portfolio))
        sim_returns, sim_vol, sim_sharpe = np.zeros((N_sim,1)), np.zeros((N_sim,1)), np.zeros((N_sim,1))
        for i in tqdm(range(N_sim)):
                weight = np.array(np.random.random(size_of_portfolio))
                exp_return, exp_vol, sharpe = sharpe_ratio_portfolio(data, weight, risk_free_rate)
                weights = weight / np.sum(weight)
                sim_weights[i, :] = weights
                sim_returns[i, :] = exp_return
                sim_vol[i, :] = exp_vol
                sim_sharpe[i, :] = sharpe
        global_result = np.concatenate((sim_weights, sim_returns, sim_vol, sim_sharpe), axis=1)
        if not as_df:
                return global_result
        else:
                result_df = pd.DataFrame(
                        data=global_result,
                        columns=["W1", "W2", "W3", "W4", "Return", "Volatility", "Sharpe"]
                )
                return result_df

In [6]:
def MC_sim_summary(data: pd.DataFrame, risk_free_rate: float, N_sim: int):
    all_sim = MC_weight_sim(data, risk_free_rate, N_sim, True)
    max_sharpe = all_sim.loc[all_sim["Sharpe"] == all_sim["Sharpe"].max()]
    min_volatility = all_sim.loc[all_sim["Volatility"] == all_sim["Volatility"].min()]
    print(max_sharpe)
    print(min_volatility)

In [7]:
MC_sim_summary(get_financial_data(["SPY AAPL MSFT AMZN"], "1y", "1d"), 0.04, 100)

[*********************100%***********************]  4 of 4 completed


100%|██████████| 100/100 [00:00<00:00, 372.88it/s]

          W1        W2        W3        W4    Return  Volatility    Sharpe
62  0.283686  0.207779  0.292231  0.216305 -0.007813    0.032532 -1.469728
          W1        W2        W3        W4    Return  Volatility     Sharpe
25  0.051194  0.402146  0.184475  0.362185 -0.000754    0.003062 -13.308075
